<a href="https://colab.research.google.com/github/peterfarrell12/BTTS_Project/blob/master/Premier_League_Data_Cleaning_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install datapackage
!pip install pandas

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
import pandas as pd
import re
from datetime import date
import calendar

In [0]:
data = pd.read_excel("PL_Game_Data_1.xlsx",)
table_data = pd.read_excel("PL_Table_Data.xlsx")

In [0]:
def get_date(season_col,date_col):
             
  y = str(season_col)
  years = y.split("/")
  y1 = years[0]
  y2 = years[1]
  date = date_col[4:]
  date = date.replace("]", "/")
  date = date.replace(")", "/")
  if "Aug" in date or "Sep" in date or "Oct" in date or "Nov" in date or "Dec" in date:
    date = date+y1
  else:
    date=date+y2

  return date
  

In [0]:
def get_week_day(date):
  
  my_date = date
  my_date = calendar.day_name[my_date.weekday()]
  return my_date

In [0]:
def get_ht(desc_col):

  nums = ["0","1","2","3","4","5","6","7","8","9",]
  ht =""

  for x in desc_col:
    if x in nums:
      i = desc_col.index(x) - 1
      ht = desc_col[:i]
  ht = re.sub(r"\d+", "", ht)
  return ht.strip()

In [0]:
def get_at(desc_col):
  nums = ["0","1","2","3","4","5","6","7","8","9",]
  at =""


  for x in desc_col:
    if x in nums:
      i = desc_col.index(x) - 1
      at = desc_col[i:]
  at = re.sub(r"\d+", "", at)
  at = re.sub(r'-', "", at)
  at = at.strip()
  return at.strip()

In [0]:
def result(desc_col):
  sep = "-"
  htg = 0
  atg = 0
  result = 0 #Home = 1, Draw = 0, Away = 2

  for x in desc_col:
    if x == sep:
      i = desc_col.index(x) -1
      j = desc_col.index(x) + 1
      htg = desc_col[i]
      atg = desc_col[j]
  if htg > atg:
    result = 1
  elif atg > htg:
    result = 2
  else:
    result = 0

  return result

In [0]:
def get_team_position(team, gameweek,season, t_d):

  ht_pos = 0

  for index, row in t_d.iterrows():
    if season.strip() == row['Season'].strip() and team.strip() == row["Club"].strip() and gameweek == (int(row["MatchDay"]) +1) :
      ht_pos = row["Position"]

  return ht_pos

In [0]:
def replace_teams(team):
  team = re.sub(r'FC', "", team)
  team = re.sub(r'&', "and", team)
  team =  re.sub(r"\b[a-zA-Z]\b", "", team)
  return team.strip()

In [0]:
def preprocess_data(sheet, td):


    sheet['Date'] = sheet.apply(lambda x: get_date(x["Season"],x["date_col"]),axis=1)
    sheet['Date'] = pd.to_datetime(sheet['Date'])
    sheet['Gameday'] = sheet.apply(lambda x: get_week_day(x['Date']), axis=1)
    sheet["Result"] = sheet.apply(lambda x: result(x["desc"]), axis = 1)
    sheet["Home Team"] = sheet.apply(lambda x: get_ht(x["desc"]), axis = 1)
    sheet["Away Team"] = sheet.apply(lambda x: get_at(x["desc"]), axis = 1)
    sheet['Home Team'] = sheet.apply(lambda x: replace_teams(x['Home Team']), axis = 1)
    sheet['Away Team'] = sheet.apply(lambda x: replace_teams(x['Away Team']), axis = 1)
    #sheet['HT Position'] = sheet.apply(lambda x: get_team_position(x["Home Team"], x["Gameweek"],x["Season"], table_data), axis=1)
    #sheet['AT Position'] = sheet.apply(lambda x: get_team_position(x["Away Team"], x["Gameweek"],x["Season"], table_data), axis=1)


    new_sheet = sheet[[ "Season","Gameweek","Gameday", "Home Team", "Away Team", "Result"]]

    return new_sheet


In [0]:
test_sheet = preprocess_data(data, table_data)

In [0]:
def get_table_position(sheet, td):
  sheet['HT Position'] = sheet.apply(lambda x: get_team_position(x["Home Team"], x["Gameweek"],x["Season"], table_data), axis=1)
  sheet['AT Position'] = sheet.apply(lambda x: get_team_position(x["Away Team"], x["Gameweek"],x["Season"], table_data), axis=1)

  new_sheet = sheet[[ "Season","Gameweek","Gameday", "Home Team", "Away Team", 'HT Position', 'AT Position', "Result"]]

  return new_sheet

In [0]:
final_sheet = get_table_position(test_sheet, table_data)

In [0]:
test_sheet['AT Position'] = test_sheet['AT Position'].astype(int)
test_sheet['HT Position'] = test_sheet['HT Position'].astype(int)

In [0]:
test_sheet.info()

In [0]:
test_sheet.describe()

In [0]:
test_sheet['Home Team'].value_counts()

In [0]:
from google.colab import files

data = final_sheet.to_csv('pl_data.csv')



In [0]:
files.download('pl_data.csv')